## Goal:
As a first and last step of cleaning:
* Read classified sidewalk data from database
* Clean sidewalks
* Write the result back to the database

## Connect to Database and fetch the data using psycopg2:

In [ ]:
import psycopg2

In [ ]:
conn = psycopg2.connect(database="****", user="****", password="", host="****", port="****")
print "Opened database successfully"

* Fetch the data:  
each row including a polygon_id (b_id) and geometry (in geoJSON format) of all sidewalks within that polygon

In [ ]:
cur = conn.cursor()
cur.execute("SELECT b_id,  st_asgeojson(ST_Collect(s_geom)) from grouped_sidewalks group by b_id")
rows = cur.fetchall()

* Each row of the fetched data looks like:

## Cleaning and Plotting Functions:

In [ ]:
# Packages:
import psycopg2
import shapely
from shapely.geometry import Point, Polygon, LineString, MultiPoint
from copy import deepcopy
import fiona
import matplotlib
from matplotlib import pyplot
import mpld3
%matplotlib inline
mpld3.enable_notebook()

# Cleaning:
def lenLine(line):
    return Point(line[0]).distance(Point(line[1]))

def cosAndsine(line):
    # cos and sine made with horizon
    if lenLine(line) != 0:
        cos = float(line[1][0] - line[0][0])/float(lenLine(line))
        sine = float(line[1][1] - line[0][1])/float(lenLine(line))
        return [cos, sine]
    else:
        
        print "Warning: Following Line is changed to a Point!!! Check your tolerances"
        print line
        return [0,0]
    
def findTwoClosePointsIndex(twoLineList):
    # Takes twolines and gives the index of two close points (one from each line) 
    # in addition to the distance between them
    distances = []
 
    for i in range(2):
        for j in range(2):
            distances.append(Point(twoLineList[0][-i]).distance(Point(twoLineList[1][-j])))
            
    if distances.index(min(distances)) == 0:
            return [[0,0], min(distances)] 
    if distances.index(min(distances)) == 1:
            return [[0,1], min(distances)] 
    if distances.index(min(distances)) == 2:
            return [[1,0], min(distances)] 
    if distances.index(min(distances)) == 3:
            return [[1,1], min(distances)] 
        
def connectToAvgIfCloseEnough(twoLineList, tolClose):
    lines = deepcopy(twoLineList)
    disAndIndex = findTwoClosePointsIndex(lines)
    if disAndIndex[1] < tolClose:
        avgX = (lines[0][-disAndIndex[0][0]][0] + lines[1][-disAndIndex[0][1]][0])/float(2)
        avgY = (lines[0][-disAndIndex[0][0]][1] + lines[1][-disAndIndex[0][1]][1])/float(2)
        lines[0][-disAndIndex[0][0]] = [avgX, avgY]
        lines[1][-disAndIndex[0][1]] = [avgX, avgY]
    return lines

        
def findTwoClosePointFromgivenPointIndex(twoLineList, givenPoint):
    # Takes twolines and a given point gives the index of two closes points (one from each line)
    # a given point in addition to the distance between them
    shortDisGivenIndex = []
    for i in range(2):
        distances = []
        for j in range(2):
            distances.append(Point(twoLineList[i][-j]).distance(givenPoint))
        shortDisGivenIndex.append([distances.index(min(distances)), min(distances)])
    return shortDisGivenIndex
            

def cleanIfintesectedAlready(twoLineList, tolInt):
    lines = deepcopy(twoLineList)
    intrsct = LineString(lines[0]).intersection(LineString(lines[1]))
    if intrsct.geom_type == 'Point':
        shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intrsct)
        for i in range(2):
            if shortDisIndices[i][1] < tolInt:
                lines[i][-shortDisIndices[i][0]] = [intrsct.x, intrsct.y]
        return lines
    
    elif intrsct.geom_type == 'LineString':
        midPoint = intrsct.interpolate(intrsct.length/2)
        shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, midPoint)
        for i in range(2):
            if shortDisIndices[i][1] < tolInt:
                lines[i][-shortDisIndices[i][0]] = [midPoint.x, midPoint.y]
        return lines
    
    elif intrsct.geom_type == 'MultiPoint':
        print "Warning: intersection of Following lines is a MultiPoint:"
        print lines
        for intPoint in intrsct:
            shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intPoint)
            for i in range(2):
                if shortDisIndices[i][1] < tolInt:
                    lines[i][-shortDisIndices[i][0]] = [intPoint.x, intPoint.y]
        return lines    
def givefirstandLasttwoLines(line):
    return [ [line[0],line[1]], [line[-2], line[-1]] ]
    
    
def extendMultiSectioLinefromBothsides(line, extL):
    extLine = deepcopy(line)
    if len(line) > 2:
        firstLast = givefirstandLasttwoLines(line)
        for j in range(len(firstLast)):
            line = firstLast[j]
            cosT = cosAndsine(line)[0]
            sinT = cosAndsine(line)[1]
            extCoordsPos = [line[-j][0] + extL * cosT, line[-j][1] + extL * sinT]
            extCoordsNeg = [line[-j][0] - extL * cosT, line[-j][1] - extL * sinT]
            if Point(extCoordsPos).distance(Point(extLine[1-3*j])) > Point(extCoordsNeg).distance(Point(extLine[1-3*j])):
                extCoords = extCoordsPos
            else:
                extCoords = extCoordsNeg
            extLine[-j] = extCoords
        return extLine
    else:
        cosT = cosAndsine(line)[0]
        sinT = cosAndsine(line)[1]
        for j in range(len(line)):
            extCoordsPos = [line[-j][0] + extL * cosT, line[-j][1] + extL * sinT]
            extCoordsNeg = [line[-j][0] - extL * cosT, line[-j][1] - extL * sinT]
            if Point(extCoordsPos).distance(Point(extLine[abs(j-1)])) > Point(extCoordsNeg).distance(Point(extLine[abs(j-1)])):
                extCoords = extCoordsPos
            else:
                extCoords = extCoordsNeg
            extLine[j] = extCoords
        return extLine

def connectIfintesectedAfterExtending(twoLineList, extL ,tolInt):
    lines = deepcopy(twoLineList)
    extLine1 = extendMultiSectioLinefromBothsides(twoLineList[0], extL)
    extLine2 = extendMultiSectioLinefromBothsides(twoLineList[1], extL)
    intrsct = LineString(extLine1).intersection(LineString(extLine2))
    
    if intrsct.geom_type == 'Point':
        shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intrsct)
        if max(shortDisIndices[0][1], shortDisIndices[1][1]) < tolInt:
            lines[0][-shortDisIndices[0][0]] = [intrsct.x, intrsct.y]
            lines[1][-shortDisIndices[1][0]] = [intrsct.x, intrsct.y]
            return lines
        else:
            return connectToAvgIfCloseEnough(lines, tolClose)
    elif intrsct.geom_type == 'LineString':
        return connectToAvgIfCloseEnough(lines, tolInt) #Not by tolClose
    elif intrsct.geom_type == 'MultiPoint':
        print "Warning: intersection of Extended version of Following lines is a MultiPoint:"
        print lines
        for intPoint in intrsct:
            shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intPoint)
            if max(shortDisIndices[0][1], shortDisIndices[1][1]) < tolInt:
                lines[0][-shortDisIndices[0][0]] = [intPoint.x, intPoint.y]
                lines[1][-shortDisIndices[1][0]] = [intPoint.x, intPoint.y]
        return lines

# Work on this else i.e. the case where the extended versions meet in form
# of MultiPoints very far and connectToAvgIfCloseEnough needs to be applied     
#             else:
#                 return connectToAvgIfCloseEnough(lines, tolClose)
        
            

def cleanDanglesofAnyKind(twoLineList, extL, tolInt, tolClose):
    extline1 = extendMultiSectioLinefromBothsides(twoLineList[0], extL)
    extline2 = extendMultiSectioLinefromBothsides(twoLineList[1], extL)
    
    if LineString(twoLineList[0]).intersects(LineString(twoLineList[1])) == True:
        return cleanIfintesectedAlready(twoLineList, tolInt)
    
    elif LineString(extline1).intersects(LineString(extline2)) == True:
        return connectIfintesectedAfterExtending(twoLineList, extL ,tolInt)
    
    else:
        return connectToAvgIfCloseEnough(twoLineList, tolClose)
    
def cleanEachGroup(linesList, extL, tolInt, tolClose):
    linesListClean = deepcopy(linesList)
    for i in range(len(linesListClean)):
        for j in range(i+1, len(linesListClean)):
            twoline = [linesListClean[i], linesListClean[j]]
            twolineclean = cleanDanglesofAnyKind(twoline, extL, tolInt, tolClose)
            if twoline[0] != twolineclean[0]:
                linesListClean[i] = twolineclean[0]
            if twoline[1] != twolineclean[1]:
                linesListClean[j] = twolineclean[1]
    return linesListClean
# Plotting:
# function: plots one line (not a lineList) with first and last points with different colors
def plot_line(ax, line, color):
    #line = [(x, y),...]
    x, y = LineString(line).xy
    # plot lines between points
    ax.plot(x, y, color= color, alpha=0.7, linewidth=1, solid_capstyle='round', zorder=1)
    # Plot first and last points with different colors
#     ax.plot(x[0], y[0], 'o', color='#80F222', zorder=2)
#     ax.plot(x[-1], y[-1], 'o', color='#F22222', zorder=2)
    
# function: handle axes limits and title of a plot of a (lineList 
def handle_axesLimTitle_for_lineList(ax, lineList, title):
    #lineList = [line1, ...]
    allXList = []
    allYList = []
    for line in lineList:
        for point in line:
            allXList.append(point[0])
            allYList.append(point[1])
    XLim = [min(allXList), max(allXList)]
    YLim = [min(allYList), max(allYList)]
    XEmptySpace = int((XLim[1]-XLim[0])/5.0)
    YEmptySpace = int((YLim[1]-YLim[0])/5.0)
    if XEmptySpace == 0:
        XEmptySpace = 1
    if YEmptySpace == 0:
        YEmptySpace = 1
    Xrange = [int(XLim[0]) - XEmptySpace, int(XLim[1]) + XEmptySpace]
    Yrange = [int(YLim[0]) - YEmptySpace, int(YLim[1]) + YEmptySpace]
    ax.set_xlim(*Xrange)
    ax.set_ylim(*Yrange)
    # ax.set_yticks(list(range(*Yrange)) + [Yrange[-1]])
    # ax.set_aspect(1)
    ax.set_title(title) 

* Clean one group to see if cleaning is working:
* Note: ast.literal_eval changes text to a dictionary

In [ ]:
import ast
extL, tolInt, tolClose = 150, 35, 25
cleanSWList = []
for row in rows:
    group = ast.literal_eval(row[1])['coordinates']
    cleanGroup = cleanEachGroup(group, extL, tolInt, tolClose)
    cleanSWList.append((row[0], {'type': 'MultiLineString', 'coordinates': cleanGroup}))
    break
# Plot the results
fig = pyplot.figure(1, figsize=(10,5), dpi=90)
ax = fig.add_subplot(121)
for line in group:
    plot_line(ax, line, '#F22222')
handle_axesLimTitle_for_lineList(ax, group, "Lines")
ax = fig.add_subplot(122)
for line in cleanSWList[0][1]['coordinates']:
    plot_line(ax, line, '#F22222')
handle_axesLimTitle_for_lineList(ax, group, "Lines Clean")

* Applying the cleaning to **n** polygons which have less than **m** number of sidewalks inside them:

In [ ]:
n = 20
m = 10000
numSWbeforeCleaning = 0
numSWafterCleaning = 0
SWProcessedList = []
cleanSWList = []
for i in range(n):
    group = ast.literal_eval(rows[i][1])['coordinates']
    numSWbeforeCleaning += len(group)
    if len(group) < m:
        SWProcessedList.append((rows[i][0], {'type': 'MultiLineString', 'coordinates': group}))
        cleanGroup = cleanEachGroup(group, extL, tolInt, tolClose)
        numSWafterCleaning += len(cleanGroup)
        cleanSWList.append((rows[i][0], {'type': 'MultiLineString', 'coordinates': cleanGroup}))
    else:
        print rows[i][0],i
print "number of sidewalks before cleaning: %d" %numSWbeforeCleaning
print "number of sidewalks after cleaning: %d" %numSWafterCleaning

In [ ]:
fig = pyplot.figure(1, figsize=(10,5), dpi=90)
ax = fig.add_subplot(121)
for row in SWProcessedList:
    for line in row[1]['coordinates']:
        plot_line(ax, line, '#F22222')
# handle_axesLimTitle_for_lineList(ax, firstPol, "Lines")
ax = fig.add_subplot(122)
for element in cleanSWList:
    for line in element[1]['coordinates']:
        plot_line(ax, line, '#F22222')
# handle_axesLimTitle_for_lineList(ax, group, "Lines Clean")

## Create a table called "Amir_CleanSW" with two columns:
1. PolygonID: includes Polygon ID
2. SWwithinPolGeom: includes the geometry of all sidewalks within each polygon

In [ ]:
conn = psycopg2.connect(database="****", user="****", password="****", host="****", port="****")
print "Opened database successfully"
cur = conn.cursor()
# Delete the table if already exist
cur.execute('''DROP TABLE IF EXISTS Amir_CleanSW''')
# Create table CleanSW
cur.execute('''CREATE TABLE Amir_CleanSW (PolygonID INT, SWwithinPolGeom geometry(MULTILINESTRING, 2926));''')
print "Table created"
conn.commit()
conn.close()

## Put clean data into the created table:

In [ ]:
conn = psycopg2.connect(database="****", user="****", password="****", host="****", port="****")
print "Opened database successfully"
cur = conn.cursor()
cur.execute('''TRUNCATE Amir_CleanSW''')

for row in cleanSWList:
    cur = conn.cursor()

    cur.execute("INSERT INTO Amir_CleanSW (PolygonID, SWwithinPolGeom) \
                VALUES (%s, ST_SetSRID(ST_GeomFromGeoJSON(%s),2926))", (row[0], str(row[1])) );

    conn.commit()
conn.close()
print "Clean Data inserted successfully"

## Code:

In [ ]:
# Packages:
import psycopg2
import shapely
from shapely.geometry import Point, Polygon, LineString, MultiPoint
from copy import deepcopy
import fiona
import matplotlib
from matplotlib import pyplot
import mpld3
%matplotlib inline
mpld3.enable_notebook()

# Cleaning:
def lenLine(line):
    return Point(line[0]).distance(Point(line[1]))

def cosAndsine(line):
    # cos and sine made with horizon
    if lenLine(line) != 0:
        cos = float(line[1][0] - line[0][0])/float(lenLine(line))
        sine = float(line[1][1] - line[0][1])/float(lenLine(line))
        return [cos, sine]
    else:
        
        print "Warning: Following Line is changed to a Point!!! Check your tolerances"
        print line
        return [0,0]
    
def findTwoClosePointsIndex(twoLineList):
    # Takes twolines and gives the index of two close points (one from each line) 
    # in addition to the distance between them
    distances = []
 
    for i in range(2):
        for j in range(2):
            distances.append(Point(twoLineList[0][-i]).distance(Point(twoLineList[1][-j])))
            
    if distances.index(min(distances)) == 0:
            return [[0,0], min(distances)] 
    if distances.index(min(distances)) == 1:
            return [[0,1], min(distances)] 
    if distances.index(min(distances)) == 2:
            return [[1,0], min(distances)] 
    if distances.index(min(distances)) == 3:
            return [[1,1], min(distances)] 
        
def connectToAvgIfCloseEnough(twoLineList, tolClose):
    lines = deepcopy(twoLineList)
    disAndIndex = findTwoClosePointsIndex(lines)
    if disAndIndex[1] < tolClose:
        avgX = (lines[0][-disAndIndex[0][0]][0] + lines[1][-disAndIndex[0][1]][0])/float(2)
        avgY = (lines[0][-disAndIndex[0][0]][1] + lines[1][-disAndIndex[0][1]][1])/float(2)
        lines[0][-disAndIndex[0][0]] = [avgX, avgY]
        lines[1][-disAndIndex[0][1]] = [avgX, avgY]
    return lines

        
def findTwoClosePointFromgivenPointIndex(twoLineList, givenPoint):
    # Takes twolines and a given point gives the index of two closes points (one from each line)
    # a given point in addition to the distance between them
    shortDisGivenIndex = []
    for i in range(2):
        distances = []
        for j in range(2):
            distances.append(Point(twoLineList[i][-j]).distance(givenPoint))
        shortDisGivenIndex.append([distances.index(min(distances)), min(distances)])
    return shortDisGivenIndex
            

def cleanIfintesectedAlready(twoLineList, tolInt):
    lines = deepcopy(twoLineList)
    intrsct = LineString(lines[0]).intersection(LineString(lines[1]))
    if intrsct.geom_type == 'Point':
        shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intrsct)
        for i in range(2):
            if shortDisIndices[i][1] < tolInt:
                lines[i][-shortDisIndices[i][0]] = [intrsct.x, intrsct.y]
        return lines
    
    elif intrsct.geom_type == 'LineString':
        midPoint = intrsct.interpolate(intrsct.length/2)
        shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, midPoint)
        for i in range(2):
            if shortDisIndices[i][1] < tolInt:
                lines[i][-shortDisIndices[i][0]] = [midPoint.x, midPoint.y]
        return lines
    
    elif intrsct.geom_type == 'MultiPoint':
        print "Warning: intersection of Following lines is a MultiPoint:"
        print lines
        for intPoint in intrsct:
            shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intPoint)
            for i in range(2):
                if shortDisIndices[i][1] < tolInt:
                    lines[i][-shortDisIndices[i][0]] = [intPoint.x, intPoint.y]
        return lines    
def givefirstandLasttwoLines(line):
    return [ [line[0],line[1]], [line[-2], line[-1]] ]
    
    
def extendMultiSectioLinefromBothsides(line, extL):
    extLine = deepcopy(line)
    if len(line) > 2:
        firstLast = givefirstandLasttwoLines(line)
        for j in range(len(firstLast)):
            line = firstLast[j]
            cosT = cosAndsine(line)[0]
            sinT = cosAndsine(line)[1]
            extCoordsPos = [line[-j][0] + extL * cosT, line[-j][1] + extL * sinT]
            extCoordsNeg = [line[-j][0] - extL * cosT, line[-j][1] - extL * sinT]
            if Point(extCoordsPos).distance(Point(extLine[1-3*j])) > Point(extCoordsNeg).distance(Point(extLine[1-3*j])):
                extCoords = extCoordsPos
            else:
                extCoords = extCoordsNeg
            extLine[-j] = extCoords
        return extLine
    else:
        cosT = cosAndsine(line)[0]
        sinT = cosAndsine(line)[1]
        for j in range(len(line)):
            extCoordsPos = [line[-j][0] + extL * cosT, line[-j][1] + extL * sinT]
            extCoordsNeg = [line[-j][0] - extL * cosT, line[-j][1] - extL * sinT]
            if Point(extCoordsPos).distance(Point(extLine[abs(j-1)])) > Point(extCoordsNeg).distance(Point(extLine[abs(j-1)])):
                extCoords = extCoordsPos
            else:
                extCoords = extCoordsNeg
            extLine[j] = extCoords
        return extLine

def connectIfintesectedAfterExtending(twoLineList, extL ,tolInt):
    lines = deepcopy(twoLineList)
    extLine1 = extendMultiSectioLinefromBothsides(twoLineList[0], extL)
    extLine2 = extendMultiSectioLinefromBothsides(twoLineList[1], extL)
    intrsct = LineString(extLine1).intersection(LineString(extLine2))
    
    if intrsct.geom_type == 'Point':
        shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intrsct)
        if max(shortDisIndices[0][1], shortDisIndices[1][1]) < tolInt:
            lines[0][-shortDisIndices[0][0]] = [intrsct.x, intrsct.y]
            lines[1][-shortDisIndices[1][0]] = [intrsct.x, intrsct.y]
            return lines
        else:
            return connectToAvgIfCloseEnough(lines, tolClose)
    elif intrsct.geom_type == 'LineString':
        return connectToAvgIfCloseEnough(lines, tolInt) #Not by tolClose
    elif intrsct.geom_type == 'MultiPoint':
        print "Warning: intersection of Extended version of Following lines is a MultiPoint:"
        print lines
        for intPoint in intrsct:
            shortDisIndices = findTwoClosePointFromgivenPointIndex(lines, intPoint)
            if max(shortDisIndices[0][1], shortDisIndices[1][1]) < tolInt:
                lines[0][-shortDisIndices[0][0]] = [intPoint.x, intPoint.y]
                lines[1][-shortDisIndices[1][0]] = [intPoint.x, intPoint.y]
        return lines

# Work on this else i.e. the case where the extended versions meet in form
# of MultiPoints very far and connectToAvgIfCloseEnough needs to be applied     
#             else:
#                 return connectToAvgIfCloseEnough(lines, tolClose)
        
            

def cleanDanglesofAnyKind(twoLineList, extL, tolInt, tolClose):
    extline1 = extendMultiSectioLinefromBothsides(twoLineList[0], extL)
    extline2 = extendMultiSectioLinefromBothsides(twoLineList[1], extL)
    
    if LineString(twoLineList[0]).intersects(LineString(twoLineList[1])) == True:
        return cleanIfintesectedAlready(twoLineList, tolInt)
    
    elif LineString(extline1).intersects(LineString(extline2)) == True:
        return connectIfintesectedAfterExtending(twoLineList, extL ,tolInt)
    
    else:
        return connectToAvgIfCloseEnough(twoLineList, tolClose)
    
def cleanEachGroup(linesList, extL, tolInt, tolClose):
    linesListClean = deepcopy(linesList)
    for i in range(len(linesListClean)):
        for j in range(i+1, len(linesListClean)):
            twoline = [linesListClean[i], linesListClean[j]]
            twolineclean = cleanDanglesofAnyKind(twoline, extL, tolInt, tolClose)
            if twoline[0] != twolineclean[0]:
                linesListClean[i] = twolineclean[0]
            if twoline[1] != twolineclean[1]:
                linesListClean[j] = twolineclean[1]
    return linesListClean
# Plotting:
# function: plots one line (not a lineList) with first and last points with different colors
def plot_line(ax, line, color):
    #line = [(x, y),...]
    x, y = LineString(line).xy
    # plot lines between points
    ax.plot(x, y, color= color, alpha=0.7, linewidth=1, solid_capstyle='round', zorder=1)
    # Plot first and last points with different colors
#     ax.plot(x[0], y[0], 'o', color='#80F222', zorder=2)
#     ax.plot(x[-1], y[-1], 'o', color='#F22222', zorder=2)
    
# function: handle axes limits and title of a plot of a (lineList 
def handle_axesLimTitle_for_lineList(ax, lineList, title):
    #lineList = [line1, ...]
    allXList = []
    allYList = []
    for line in lineList:
        for point in line:
            allXList.append(point[0])
            allYList.append(point[1])
    XLim = [min(allXList), max(allXList)]
    YLim = [min(allYList), max(allYList)]
    XEmptySpace = int((XLim[1]-XLim[0])/5.0)
    YEmptySpace = int((YLim[1]-YLim[0])/5.0)
    if XEmptySpace == 0:
        XEmptySpace = 1
    if YEmptySpace == 0:
        YEmptySpace = 1
    Xrange = [int(XLim[0]) - XEmptySpace, int(XLim[1]) + XEmptySpace]
    Yrange = [int(YLim[0]) - YEmptySpace, int(YLim[1]) + YEmptySpace]
    ax.set_xlim(*Xrange)
    ax.set_ylim(*Yrange)
    # ax.set_yticks(list(range(*Yrange)) + [Yrange[-1]])
    # ax.set_aspect(1)
    ax.set_title(title) 

In [ ]:
import psycopg2
conn = psycopg2.connect(database="****", user="****", password="", host="****", port="****")
print "Opened database successfully"
cur = conn.cursor()
cur.execute("SELECT b_id,  st_asgeojson(ST_Collect(s_geom)) from yun_grouped_sidewalks group by b_id")
rows = cur.fetchall()
print "Data Fetched successfully"

In [ ]:
n =  len(rows)-1
m = 1000
extL, tolInt, tolClose = 180, 80, 25
numSWbeforeCleaning = 0
numSWafterCleaning = 0
SWProcessedList = []
cleanSWList = []
for i in range(n):
    group = ast.literal_eval(rows[i][1])['coordinates']
    numSWbeforeCleaning += len(group)
    if len(group) < m:
        SWProcessedList.append((rows[i][0], {'type': 'MultiLineString', 'coordinates': group}))
        cleanGroup = cleanEachGroup(group, extL, tolInt, tolClose)
        numSWafterCleaning += len(cleanGroup)
        cleanSWList.append((rows[i][0], {'type': 'MultiLineString', 'coordinates': cleanGroup}))
    else:
        print rows[i][0],i
print "number of sidewalks before cleaning: %d" %numSWbeforeCleaning
print "number of sidewalks after cleaning: %d" %numSWafterCleaning

In [ ]:
print len(cleanSWList)

In [ ]:
conn = psycopg2.connect(database="****", user="****", password="****", host="****", port="****")
print "Opened database successfully"
cur = conn.cursor()
# Delete the table if already exist
cur.execute('''DROP TABLE IF EXISTS Amir_CleanSW_180_80_25''')
# Create table CleanSW
cur.execute('''CREATE TABLE Amir_CleanSW_180_80_25 (PolygonID INT, SWwithinPolGeom geometry(MULTILINESTRING, 2926));''')
print "Table created"
conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect(database="****", user="****", password="****", host="****", port="****")
print "Opened database successfully"
cur = conn.cursor()
cur.execute('''TRUNCATE Amir_CleanSW_180_80_25''')

for row in cleanSWList:
    cur = conn.cursor()
    cur.execute("INSERT INTO Amir_CleanSW_180_80_25 (PolygonID, SWwithinPolGeom) \
                VALUES (%s, ST_SetSRID(ST_GeomFromGeoJSON(%s),2926))", (row[0], str(row[1])) );

    
conn.commit()
conn.close()
print "Clean Data inserted successfully"